# Homework 9
# Done By: Hiba Jalloul and Pankaj Chouhan

In [564]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from scipy.stats import multivariate_normal
from random import random, randint

# EM 

In [378]:
def EM_init (x, it):
    n = len(x)
    mu1 = np.zeros((it,2))
    mu2 = np.zeros((it,2))
    sigma1 = np.zeros((it,2,2))
    sigma2 = np.zeros((it,2,2))
    pi1 = np.zeros(it)
    pi2 = np.zeros(it)

    kmeans = KMeans(n_clusters=2, random_state=0).fit(x)
    y = kmeans.predict(x)

    pi1[0] = len(y[y==0])/n
    pi2[0] =  len(y[y==1])/n
    
    mu1[0] = kmeans.cluster_centers_[0]
    mu2[0] = kmeans.cluster_centers_[1]
    
    sigma1[0] = np.cov(x[y==0],rowvar=False)
    sigma2[0] = np.cov(x[y==1],rowvar=False)
    
    return pi1, pi2, mu1, mu2, sigma1, sigma2

def EM(x, pi1, pi2, mu1, mu2, sigma1, sigma2):
    n = len(x)
    ll = {}
    ll[0] = sum(np.log( pi1[0]*multivariate_normal.pdf(x,mu1[0],sigma1[0]) + pi2[0]*multivariate_normal.pdf(x,mu2[0],sigma2[0]) ))

    for i in range(1,len(mu1)):
        # Expectation Step - Conditional Probabilities
        y1 = (pi1[i-1]*multivariate_normal.pdf(x,mu1[i-1],sigma1[i-1]))/(pi1[i-1]*multivariate_normal.pdf(x,mu1[i-1],sigma1[i-1]) + pi2[i-1]*multivariate_normal.pdf(x,mu2[i-1],sigma2[i-1]))
        y2 = (pi2[i-1]*multivariate_normal.pdf(x,mu2[i-1],sigma2[i-1]))/(pi1[i-1]*multivariate_normal.pdf(x,mu1[i-1],sigma1[i-1]) + pi2[i-1]*multivariate_normal.pdf(x,mu2[i-1],sigma2[i-1]))

        # Maximization Step
        pi1[i] = sum(y1) / n
        pi2[i] = sum(y2) / n

        mu1[i] = sum(y1[:,None]*x) / sum(y1)
        mu2[i] = sum(y2[:,None]*x) / sum(y2)
        
        sigma1[i] = np.transpose(np.multiply(x-mu1[i],np.transpose(np.mat(y1)))) * (x-mu1[i]) / sum(y1)
        sigma2[i] = np.transpose(np.multiply(x-mu2[i],np.transpose(np.mat(y2)))) * (x-mu2[i]) / sum(y2)

        # Loglikelihood
        ll[i] = sum(np.log( pi1[i]*multivariate_normal.pdf(x,mu1[i],sigma1[i]) + pi2[i]*multivariate_normal.pdf(x,mu2[i],sigma2[i]) ))
        if abs(ll[i]-ll[i-1])<= 1e-6:
            break
    print('Number of iterations till convergence = ',i)
    print('Pi1 = ' ,pi1[i], '\nPi2 = ', pi2[i],'\nMU1 = ', mu1[i],'\nMU2 = ', mu2[i],'\nCov1 = ', sigma1[i],'\nCov2 = ', sigma2[i]) 

## 1- a) xeasy Data - EM

In [386]:
x = pd.read_csv('xeasy.txt', header = None).to_numpy()
param = EM_init(x,100)
EM(x, param[0],param[1],param[2],param[3],param[4],param[5])

Number of iterations till convergence =  11
Pi1 =  0.5911645391966039 
Pi2 =  0.4088354608033957 
MU1 =  [ 3.01884339 -0.17709437] 
MU2 =  [0.02845147 3.07054136] 
Cov1 =  [[1.00530907 0.16004762]
 [0.16004762 0.94212979]] 
Cov2 =  [[ 1.01846298 -0.0585727 ]
 [-0.0585727   0.95453142]]


## 1- b) x1 Data - EM

In [390]:
x = pd.read_csv('x1.txt', header = None).to_numpy()
param = EM_init(x,150)
EM(x, param[0],param[1],param[2],param[3],param[4],param[5])

Number of iterations till convergence =  128
Pi1 =  0.652531726795146 
Pi2 =  0.347468273204854 
MU1 =  [-0.10092945  1.97348592] 
MU2 =  [2.08399969 0.12486097] 
Cov1 =  [[1.70370708 0.14914392]
 [0.14914392 2.19983907]] 
Cov2 =  [[ 0.86282012 -0.03909889]
 [-0.03909889  1.05880181]]


## 1- c) x2 Data - EM

In [388]:
x = pd.read_csv('x2.txt', header = None).to_numpy()
param = EM_init(x,100)
EM(x, param[0],param[1],param[2],param[3],param[4],param[5])

Number of iterations till convergence =  63
Pi1 =  0.5120415558210427 
Pi2 =  0.4879584441789575 
MU1 =  [ 0.01426876 -0.04650351] 
MU2 =  [ 0.17809457 -0.1252182 ] 
Cov1 =  [[1.06284512 0.06639239]
 [0.06639239 0.88839857]] 
Cov2 =  [[9.25900786 0.76787325]
 [0.76787325 9.41547704]]


# Provable EM

In [ ]:
def ProvableEM (x, l, it):
    pi, mu, sigma = PEM_init(x, l, it)
    pi, mu, sigma = PEM(x, pi, mu, sigma)
    pi, mu, sigma = Pruning(pi, mu, sigma)
    pi1, pi2, mu1, mu2, sigma1, sigma2 = Pruning_to_EM(it, pi, mu, sigma)
    EM(x, pi1, pi2, mu1, mu2, sigma1, sigma2)
    
    
def PEM_init (x, l, it):
    seed(1)
    pi = np.zeros((l,it))
    pi[:,0] = 1/l

    mu = np.zeros((l,it,2))
    for i in range(l):
        mu[i,0,0] = random()
        mu[i,0,1] = random()
        
    sigma = np.zeros((l,it,2,2))
    for i in range(l):
        norms = []
        for j in range(l):
            norms.append(np.linalg.norm(mu[i][0]-mu[j][0]))
        sigma[i][0][0][0] = sorted(norms)[1]
        sigma[i][0][1][1] = sorted(norms)[1]
    
    return pi, mu, sigma


def PEM(x, pi, mu, sigma):
    n = len(x)  #Number of observations
    l = len(pi)  #Number of clusters
    it = len(pi[0])   #Number of iterations
    ll = {}
    
    # Initial Loglikelihood
    a = 0
    for j in range(l):
        a += pi[j][0]*multivariate_normal.pdf(x,mu[j][0],sigma[j][0])
    ll[0] = sum(np.log(a))

    for i in range(1,it):
        
        # Expectation Step - Conditional Probabilities
        d = 0
        for k in range(l):
            d += pi[k][i-1]*multivariate_normal.pdf(x,mu[k][i-1],sigma[k][i-1])  
        y = []
        for j in range(l):
            y.append((pi[j][i-1]*multivariate_normal.pdf(x,mu[j][i-1],sigma[j][i-1]))/d)
        
        # Maximization Step
        for j in range(l):
            pi[j][i] = sum(y[j])/n
            mu[j][i] = sum(y[j][:,None]*x) / sum(y[j])
            sigma[j][i] = np.transpose(np.multiply(x-mu[j][i],np.transpose(np.mat(y[j])))) * (x-mu[j][i]) / sum(y[j])
        
        # Loglikelihood
        a = 0
        for j in range(l):
            a += pi[j][i]*multivariate_normal.pdf(x,mu[j][i],sigma[j][i])
        ll[i] = sum(np.log(a))

        if abs(ll[i]-ll[i-1])<= 1e-6:
            break

    return pi[:,i], mu[:,i], sigma[:,i]


def Pruning (pi, mu, sigma):
    l = len(pi) #initial number of clusters
    
    # Removing clusters with small pi
    keep = pi>= (1/(4*l)) #clusters to be kept
    pi = pi[keep]
    mu = mu[keep]
    sigma = sigma[keep]
    
    #Select k centers furthest from each other
    l = len(pi)  #updated number of clusters
    first = randint(0,l-1)  #Select one random cluster
    norms = []
    for j in range(l):
        norms.append(np.linalg.norm(mu[first]-mu[j]))
    d = sorted(norms)[l-1]
    second = norms.index(d)   #Select the second cluster with the maximum distance to the 1st one
    keep = [first,second]    #clusters to be kept
    pi = pi[keep]
    mu = mu[keep]
    sigma = sigma[keep]

    return pi, mu, sigma
    
    
def Pruning_to_EM(it,pi,mu,sigma):
    mu1 = np.zeros((it,2))
    mu2 = np.zeros((it,2))
    sigma1 = np.zeros((it,2,2))
    sigma2 = np.zeros((it,2,2))
    pi1 = np.zeros(it)
    pi2 = np.zeros(it)

    pi1[0] = pi[0]
    pi2[0] = pi[1]

    mu1[0] = mu[0]
    mu2[0] = mu[1]

    sigma1[0] = sigma[0]
    sigma2[0] = sigma[1]

    return pi1, pi2, mu1, mu2, sigma1, sigma2

## 2- a) xeasy Data - Provable EM

In [616]:
x = pd.read_csv('xeasy.txt', header = None).to_numpy()
ProvableEM (x, 8, 50)

Number of iterations till convergence =  25
Pi1 =  0.5911508303123365 
Pi2 =  0.4088491696876623 
MU1 =  [ 3.01887781 -0.17712928] 
MU2 =  [0.02850196 3.07048294] 
Cov1 =  [[1.00526613 0.16009589]
 [0.16009589 0.9420875 ]] 
Cov2 =  [[ 1.01852677 -0.0586482 ]
 [-0.0586482   0.95461792]]


## 2- b) x1 Data - Provable EM

In [622]:
x = pd.read_csv('x1.txt', header = None).to_numpy()
ProvableEM (x, 8, 150)

Number of iterations till convergence =  57
Pi1 =  0.3470288143774014 
Pi2 =  0.6529711856225996 
MU1 =  [2.08485114 0.12385531] 
MU2 =  [-0.09991148  1.97277624] 
Cov1 =  [[ 0.86226216 -0.0387252 ]
 [-0.0387252   1.0577065 ]] 
Cov2 =  [[1.70467175 0.14810574]
 [0.14810574 2.1999761 ]]


## 2- c) x2 Data - Provable EM

In [619]:
x = pd.read_csv('x2.txt', header = None).to_numpy()
ProvableEM (x, 8, 100)

Number of iterations till convergence =  55
Pi1 =  0.4881095760437188 
Pi2 =  0.5118904239562811 
MU1 =  [ 0.1780944 -0.1251714] 
MU2 =  [ 0.01422056 -0.0465249 ] 
Cov1 =  [[9.25715867 0.76776777]
 [0.76776777 9.4131219 ]] 
Cov2 =  [[1.06218068 0.06628234]
 [0.06628234 0.88813193]]
